The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod).

# Stage 3: Deploy the Model in Wallaroo
 
In this stage, we upload the trained model and the processing steps to Wallaroo, then set up and deploy the inference pipeline. 

Once deployed we can feed the newest batch of data to the pipeline, do the inferences and write the results to a results table.

For clarity in this demo, we have split the training/upload task into two notebooks:

* `02_automated_training_process.ipynb`: Train and pickle ML model.
* `03_deploy_model.ipynb`: Upload the model to Wallaroo and deploy into a pipeline.

Assuming no changes are made to the structure of the model, these two notebooks, or a script based on them, can then be scheduled to run on a regular basis, to refresh the model with more recent training data and update the inference pipeline.

This notebook is expected to run within the Wallaroo instance's Jupyter Hub service to provide access to all required Wallaroo libraries and functionality.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

The process of uploading the model to Wallaroo follows these steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and set up the workspace.
* [Upload The Model](#upload-the-model): Upload the model and autoconvert for use in the Wallaroo engine.
* [Upload the Processing Modules](#upload-the-processing-modules): Upload the processing modules.
* [Create and Deploy the Pipeline](#create-and-deploy-the-pipeline): Create the pipeline with the model and processing modules as steps, then deploy it.
* [Test the Pipeline](#test-the-pipeline): Verify that the pipeline works with the sample data.

### Connect to Wallaroo

First we import the required libraries to connect to the Wallaroo instance, then connect to the Wallaroo instance.

In [37]:
import json
import pickle
import pandas as pd
import numpy as np
import pyarrow as pa

import simdb # module for the purpose of this demo to simulate pulling data from a database

# from wallaroo.ModelConversion import ConvertXGBoostArgs, ModelConversionSource, ModelConversionInputType
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [38]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [39]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [40]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

The workspace `housepricing` will either be created or, if already existing, used and set to the current workspace.

In [41]:
new_workspace = get_workspace(workspace_name)
new_workspace

{'name': 'housepricing', 'id': 12, 'archived': False, 'created_by': 'fa240e86-5fb9-4d58-bf5d-f82bf0b2b172', 'created_at': '2023-09-05T18:08:39.509632+00:00', 'models': [{'name': 'housepricemodel', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 5, 18, 40, 36, 492074, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 5, 18, 8, 42, 560324, tzinfo=tzutc())}, {'name': 'postprocess', 'versions': 3, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 5, 18, 44, 48, 405551, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 5, 18, 8, 47, 58060, tzinfo=tzutc())}], 'pipelines': [{'name': 'housing-pipe', 'create_time': datetime.datetime(2023, 9, 5, 18, 8, 50, 854139, tzinfo=tzutc()), 'definition': '[]'}]}

In [42]:
_ = wl.set_current_workspace(new_workspace)

### Upload The Model

With the connection set and workspace prepared, upload the model created in `02_automated_training_process.ipynb` into the current workspace.

In [43]:
hpmodel = wl.upload_model(model_name, model_file, framework=wallaroo.framework.Framework.ONNX).configure()

## Upload the Processing Modules

Upload the `postprocess.py` modules as models to be added to the pipeline.

In [44]:
# load the postprocess module

input_schema = pa.schema([
    pa.field('variable', pa.list_(pa.float64()))
])


output_schema = pa.schema([
    pa.field('variable', pa.list_(pa.float64()))
])



module_post = (wl.upload_model("postprocess", 
                              "./postprocess.py", 
                              framework=wallaroo.framework.Framework.PYTHON)
                              .configure('python',
                                         input_schema=input_schema,
                                         output_schema=output_schema)
                )

### Create and Deploy the Pipeline

Create the pipeline with the preprocess module, housing model, and postprocess module as pipeline steps, then deploy the newpipeline.

In [45]:
pipeline = get_pipeline(pipeline_name)

pipeline.clear()

# pipeline.add_model_step(module_pre)
pipeline.add_model_step(hpmodel)
pipeline.add_model_step(module_post)

pipeline.deploy()

name,housing-pipe
created,2023-09-05 18:08:50.854139+00:00
last_updated,2023-09-05 18:46:52.054084+00:00
deployed,True
tags,
versions,"8f199fb9-cb4b-4e2b-9a91-35c002016018, dc3351f3-408d-46a4-b761-d22d24f83a2b, 6006adf0-3244-4490-9fd6-229a86c72a73, c81ee0b1-bb26-40ea-bba0-c1eab788f41f, fa1f1348-0b8a-46c7-8c4c-dde666a2ca41"
steps,housepricemodel
published,False


### Test the Pipeline

We will use a single query from the simulated `housing_price` table and infer.  When successful, we will undeploy the pipeline to restore the resources back to the Kubernetes environment.

In [46]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} limit 1"
print(query)

# read in the data
singleton = pd.read_sql_query(query, conn)
conn.close()

singleton.loc[:, ["id", "date", "list_price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot"]]

_vars = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view',
'condition', 'grade', 'sqft_above', 'sqft_basement', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'house_age', 'renovated', 'yrs_since_reno']

def create_features(housing_data):
    thisyear = datetime.datetime.now().year
    housing_data['house_age'] = thisyear - housing_data['yr_built']
    housing_data['renovated'] =  np.where((housing_data['yr_renovated'] > 0), 1, 0) 
    housing_data['yrs_since_reno'] =  np.where(housing_data['renovated'], housing_data['yr_renovated'] - housing_data['yr_built'], 0)
    return pd.DataFrame({
        "tensor": housing_data.loc[:, _vars].to_numpy().tolist()
        }
    )

evaluation_frame = create_features(singleton)
evaluation_frame

select * from house_listings limit 1


,tensor
0,"[3.0, 1.0, 1180.0, 5650.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 47.5112, -122.257, 1340.0, 5650.0, 68.0, 0.0, 0.0]"


In [47]:
result = pipeline.infer(evaluation_frame)
display(result)

,time,in.tensor,out.variable,check_failures
0,2023-09-05 18:46:56.382,"[3.0, 1.0, 1180.0, 5650.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 47.5112, -122.257, 1340.0, 5650.0, 68.0, 0.0, 0.0]",[224852.0],0


When finished, we undeploy the pipeline to return the resources back to the environment.

In [48]:
pipeline.undeploy()

name,housing-pipe
created,2023-09-05 18:08:50.854139+00:00
last_updated,2023-09-05 18:46:52.054084+00:00
deployed,False
tags,
versions,"8f199fb9-cb4b-4e2b-9a91-35c002016018, dc3351f3-408d-46a4-b761-d22d24f83a2b, 6006adf0-3244-4490-9fd6-229a86c72a73, c81ee0b1-bb26-40ea-bba0-c1eab788f41f, fa1f1348-0b8a-46c7-8c4c-dde666a2ca41"
steps,housepricemodel
published,False


With this stage complete, we can proceed to Stage 4: Regular Batch Inference.